# First steps on the GPU
**Author**: Stephan Hageboeck, CERN

## CUDA progamming in a notebook?
It is indeed a bit ununsual to program CUDA in a Python notebook. We will use a few tricks to make it work. The notebook provides a uniform environment to all participants, such that we can access the SWAN GPUs without having to worry about the operating systems and installed software every participant brings. In order to compile the cuda programs, we will
- write a notebook cell into a `.cu` file
- compile it using the nvcc compiler
- invoke the application from the notebook
- and convert the image such that the notebook can display it.

Let's first check that we have a GPU attached to the session:

In [ ]:
!nvidia-smi

---------------------------------------------------------------------------

# 1. HelloWorld example
Here we have a very basic helloWorld program that prints a message from the host. Let's convert it into a GPU kernel.

### Your tasks:
- Convert the HelloWorld function to a kernel, and call it from main().
- In the kernel, fill in the variables that print thread index and block index.
- Try a few launch configurations with more threads / more blocks.

In [ ]:
%%writefile helloWorld.cu
#include <cstdio>
#include <iostream>

// kernel definition
void HelloWorld() {
  printf("Hello world from block %d thread %d.\n", -1, -1);
}

int main() {
  const auto nBlock = 1;
  const auto nThread = 1;

  HelloWorld();

  if (auto errorCode = cudaDeviceSynchronize();
      errorCode != cudaSuccess) {
    std::cerr << "Encountered cuda error '"
      << cudaGetErrorName(errorCode)
      << "' with description: "
      << cudaGetErrorString(errorCode) << "\n";
    return 1;
  }

  return 0;
}



## Compile, execute, display
To have consistent line numbers when compiling, we use a little trick:
- We create an intermediate file where we add an extra line that accounts for the line that's occupied by the `writefile` magic above.
- We compile that file, so when you get an error, the lines numbers are like in the notebook.
We put intermediate files in a `tmp/` folder, so they don't pollute our main directory.

For the compilation step, we add `-g` to have debug symbols in the executable, `-std=c++17` for modern C++, and `-O2` to benefit from compiler optimisations on the host side.

In [ ]:
%%bash
mkdir -p tmp/
sed '1s/.*/\n\0/' < helloWorld.cu > tmp/helloWorld.cu
nvcc -I source/ tmp/helloWorld.cu -std=c++17 -g -O2 -o tmp/helloWorld

### Execute
You can now invoke the executable in `tmp/` by prepending a `!`

In [ ]:
! tmp/helloWorld

-------------------------------------


# 2. Vector Addition
In this example, we add two arrays on the host and on the device. We use timers to measure the execution speed. There's already a draft kernel that adds the two vectors with a single thread and a single block. We will now try to make this kernel much more efficient, and to fully utilise the device.

The arrays are initialised as follows:
```
x = {0,  1,  2,  3, ...}
y = {0, -1, -2, -3, ...}
```
We will run the computation
```
y[i] = x[i] + y[i]
```
once on the host and once on the device. If you do everything correctly, we would expect `y` to be
```
y = {0, 1, 2, 3, ...}
```
when you complete the task. The program will check this.

**Note**:
You don't need to understand every line of the program. Focus on the kernel, the kernel launch and the two tasks, which are marked in the source code.

If you are interested in the timer: The `Timer` struct starts a timer when it is constructed, and it stops and prints the elapsed time when it goes out of
scope. That's why the sections we want to time are in blocks delimited by `{ }`.

### Your tasks:
1. Implement an efficient grid-strided loop. Currently, every thread steps through every item.
2. Find an efficient launch configuration to fully use the device.

How fast can you make the kernel?

In [ ]:
%%writefile vectorAdd.cu
#include "Timer.h"

#include <algorithm>
#include <cstdio>
#include <iostream>
#include <numeric>

__global__
void add(int n,  int * x,  int * y)
{
  // Task 1:
  // ------------------------------------------------------
  // Set index and stride such that we can run an efficient
  // grid-strided loop

  const auto index = 0;
  const auto stride = 1;
  for (int i = index; i < n; i += stride)
    y[i] = x[i] + y[i];
}

bool checkResult(int * array, size_t N) {
  for (size_t i = 0; i < N; ++i) {
    if (array[i] != i) return false;
  }
  return true;
}

int main() {
  // This is the length of the vectors we want to add:
  const auto N = 100'000'000;

  int * x;
  int * y;
  cudaMallocManaged(&x, N * sizeof( int), cudaMemAttachHost);
  cudaMallocManaged(&y, N * sizeof( int), cudaMemAttachHost);

  // Initialise arrays as follows:
  // x = { 0,  1,  2, ... }
  // y = {-0, -1, -2, ... }
  {
    Timer timer{ "init arrays on host" };
    std::iota(x, x + N, 0);
    std::transform(x, x + N, y, [](int i){ return -1 * i; });
  }

  // Add them once. Now y should be equal to 0:
  {
    Timer timer{ "add on host" };
    for (unsigned int i = 0; i < N; ++i) {
      y[i] = x[i] + y[i];
    }
  }

  // Bring arrays to GPU.
  // Note that this step is optional, because they would automatically
  // be copied once the kernel accesses them.
  // This enables us to time copy and compute separately.
  {
    Timer timer{ "copy to device memory" };
    int currentDevice;
    cudaGetDevice(&currentDevice);
    cudaMemPrefetchAsync(x, N*sizeof(int), currentDevice);
    cudaMemPrefetchAsync(y, N*sizeof(int), currentDevice);

    if (const auto errorCode = cudaDeviceSynchronize();
        errorCode != cudaSuccess) {
      std::cerr << "When copying, encountered cuda error " << errorCode << " '"
        << cudaGetErrorName(errorCode)
        << "' with description:"
        << cudaGetErrorString(errorCode) << "\n";
      return 2;
    }
  }


  // Add them on the GPU. Now y should be {0, 1, 2, ...}
  {
    Timer timer{ "add on device" };

    // Task 2:
    // --------------------------------------------------------
    // Find an efficient launch configuration that exhausts the
    // capabilities of the device

    const auto nBlock = 1;
    const auto nThread = 1;

    add<<< nBlock , nThread >>>(N, x, y);

    if (const auto errorCode = cudaDeviceSynchronize();
        errorCode != cudaSuccess) {
      std::cerr << "Encountered cuda error '"
        << cudaGetErrorName(errorCode)
        << "' with description: "
        << cudaGetErrorString(errorCode) << "\n";
      return 1;
    }
  }

  {
    Timer timer{ "Access y array on host" };

    std::cout << "\ny[0] = " << y[0]
      << "\ny[" << N/2 << "] = " << y[N/2]
      << "\ny[" << N-1 << "] = " << y[N-1] << "\n";
  }
  if (checkResult(y, N))
    std::cout << "Addition seems to be correct.\n";
  else
    std::cout << "Addition seems to have failed.\n";

  cudaFree(x);
  cudaFree(y);

  return 0;
}


### Compile and execute
We proceed as for the helloWorld example by correcting line numbers and compiling in `tmp/`.

In [ ]:
%%bash
mkdir -p tmp/
sed '1s/.*/\n\0/' < vectorAdd.cu > tmp/vectorAdd.cu
nvcc -I source/ tmp/vectorAdd.cu -std=c++17 -g -O2 -o tmp/vectorAdd

Run the executable. How fast can you go with an optimised launch configuration?

In [ ]:
! tmp/vectorAdd

# Solution?
One possible solution can be found in [source/solution/vectorAdd.cu](source/solution/vectorAdd.cu)

In [ ]:
%load source/solution/vectorAdd.cu